In [1]:
# VERSION 1 - MARKET BASKET ANALYSIS
from google.colab import files
uploaded = files.upload()
import pandas as pd
df = pd.read_csv('Retail_Transactions_Dataset.csv', low_memory=False, keep_default_na= False)
# I. Data Cleaning
## I.1. Check Column Type
df.info()
## I.2. Check Blank/Null
df.isna().mean()*100
## I.3. Check duplicate
### For full row
"Duplicate full row:", df.duplicated().sum()
### For only 'Transaction_ID'
"Duplicate ID:", df['Transaction_ID'].duplicated().sum()
## I.4. Check Outliers: by IQR & export for preventing numbers of outliers is too high
### Columns "Total_Cost"
q1_Total_Cost = df['Total_Cost'].quantile(0.25)
q3_Total_Cost = df['Total_Cost'].quantile(0.75)
iqr_Total_Cost = q3_Total_Cost - q1_Total_Cost
lower_bound_Total_Cost = q1_Total_Cost - 1.5 * iqr_Total_Cost
upper_bound_Total_Cost = q3_Total_Cost + 1.5 * iqr_Total_Cost
outliers_Total_Cost = ((df['Total_Cost'] < lower_bound_Total_Cost) |(df['Total_Cost'] > upper_bound_Total_Cost))
'Outliers Total Cost:', outliers_Total_Cost.mean()*100
outliers_Total_Cost_list = df[outliers_Total_Cost]
outliers_Total_Cost_list.to_csv('outliers_Total_Cost_list.csv', index= False)
### Columns "Total_Items"
q1_Total_Items = df['Total_Items'].quantile(0.25)
q3_Total_Items = df['Total_Items'].quantile(0.75)
iqr_Total_Items = q3_Total_Items - q1_Total_Items
lower_bound_Total_Items = q1_Total_Items - 1.5 * iqr_Total_Items
upper_bound_Total_Items = q3_Total_Items + 1.5 * iqr_Total_Items
outliers_Total_Items = ((df['Total_Items'] < lower_bound_Total_Items) |(df['Total_Items'] > upper_bound_Total_Items))
print('Outliers Total Items:', outliers_Total_Items.mean()*100)
outliers_Total_Items_list = df[outliers_Total_Items]
outliers_Total_Items_list.to_csv('outliers_Total_Items_list.csv', index= False)

# II. Data Handling
## II.1. Change type to datetime: Column "Date"
df['Date'] = pd.to_datetime(df['Date'], errors= 'coerce')
df.info()

# III. Export cleaned data
df.to_csv('Retail_Transactions_Dataset_cleaned.csv', index= False)

# IV. MBA
## IV.1. Change dataset to "Transaction_ID – Item" type
### IV.1.1. Split Column "Product"
from ast import literal_eval
df['Product'] = df['Product'].apply(literal_eval)
df = df.sample(100000, random_state=42)
df_exploded = df.explode('Product')[['Transaction_ID', 'Product']]
df_exploded = df_exploded.reset_index(drop= True)
### IV.1.2. One-hot Encoding for Column "Product"
basket = pd.crosstab(df_exploded['Transaction_ID'], df_exploded['Product'])
basket = (basket > 0).astype(bool)
### IV.1.3. Apply Apriori Algorithm, Association Rules. Finally filter Top 10 by Support, Top 10 by Confidence, Top 10 by Lift
from mlxtend.frequent_patterns import fpgrowth, association_rules
frequent_items = fpgrowth(basket, min_support = 0.002, use_colnames= True)
rules = association_rules(frequent_items, metric= 'confidence', min_threshold = 0.05)
print("Frequent itemsets:", len(frequent_items))
print("Rules:", len(rules))
print('Top 10 by Support')
print(rules.sort_values('support', ascending= False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))
print('\nTop 10 by Confidence')
print(rules.sort_values('confidence', ascending= False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))
print('\nTop 10 by Lift')
print(rules.sort_values('lift', ascending= False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

Saving Retail_Transactions_Dataset.csv to Retail_Transactions_Dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 13 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Transaction_ID     1000000 non-null  int64  
 1   Date               1000000 non-null  object 
 2   Customer_Name      1000000 non-null  object 
 3   Product            1000000 non-null  object 
 4   Total_Items        1000000 non-null  int64  
 5   Total_Cost         1000000 non-null  float64
 6   Payment_Method     1000000 non-null  object 
 7   City               1000000 non-null  object 
 8   Store_Type         1000000 non-null  object 
 9   Discount_Applied   1000000 non-null  bool   
 10  Customer_Category  1000000 non-null  object 
 11  Season             1000000 non-null  object 
 12  Promotion          1000000 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(9)
memory usage: 9

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
